In [ ]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd
import tensorflow_probability as tfp
import tensorflow.keras.backend as K
import math as m
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import helper

In [ ]:
# this is just a visual reference for the meaning of alpha
import matplotlib.gridspec as gridspec
subplots = [plt.subplot(grid_loc) for grid_loc in gridspec.GridSpec(1, 8)]
for i,sp in enumerate(subplots):
    data = helper.generate_data(200,alpha=i*45,sigma_reward=0.001)
    sp.scatter(*data['z'].T,c=data['r'])
    sp.set_aspect('equal')
    sp.axis('off')
    sp.set_title(str(i*45))

In [ ]:
# example data
data = helper.generate_data(20,alpha=90,sigma_reward=0.001)
plt.scatter(*data['z'].T,c=data['r'])
plt.gca().set_aspect('equal')
plt.colorbar();

# labels
labels = ['{0}'.format(i) for i in range(data['z'].shape[0])]
for label, x, y in zip(labels, data['z'][:, 0], data['z'][:, 1]):
    plt.annotate(
        label,
        xy=(x, y), xytext=(-20, 20),
        textcoords='offset points', ha='right', va='bottom',
        bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))

In [ ]:
# relevant dimension of data points for alpha = 90
plt.plot(data['z'].T[1],'.-')

In [ ]:
def iterative_model_log_llhs(z, r, alpha, sigma_reward):
    llhs = np.array(
    [helper.model_log_llh(z[:t], r[:t], alpha=alpha, sigma_reward=sigma_reward) 
        for t in range(z.shape[0])])
    return llhs

In [ ]:
for alpha in np.arange(0,360,45):
    plt.plot(iterative_model_log_llhs(data['z'], data['r'], alpha=alpha, sigma_reward=0.1),'.-')
plt.legend(list(map(str,np.arange(0,360,45))))

In [ ]:
for alpha in np.arange(0,360,45):
    plt.plot(iterative_model_log_llhs(data['z'], data['r'], alpha=alpha, sigma_reward=0.5),'.-')
plt.legend(list(map(str,np.arange(0,360,45))))

In [ ]:
%%time
helper.model_log_llh(data['z'], data['r'], alpha=alpha, sigma_reward=0.1)

In [ ]:
alpha_samples = tfd.Uniform(0,180).sample(1000)
plt.hist(np.array([helper.model_log_llh(data['z'], data['r'], alpha=alpha_sample, sigma_reward=0.5) for alpha_sample in alpha_samples]))

In [ ]:
1/1e-100

In [ ]:
import helper

In [ ]:
helper.model_llh(data['z'], data['r'], alpha=90, sigma_reward=0.5)

In [ ]:
# example data
data = helper.generate_data(500,alpha=45,sigma_reward=0.001)
plt.scatter(*data['z'].T,c=data['r'])
plt.gca().set_aspect('equal')
plt.colorbar();

In [ ]:
def llh_under(z, r, alpha, sigma_reward, method='np'):
    gamma = helper.gamma_from_alpha(alpha)
    if method == 'tf':
        prob_per_sample = tfp.distributions.Normal(loc=tf.reduce_sum(tf.multiply(gamma,z),1), scale=sigma_reward).prob(r)
    elif method == 'np':
        prob_per_sample = tf.exp(-0.5 * 
                (tf.reduce_sum(tf.multiply(gamma,z),1) - r)**2 / sigma_reward**2) / (2*np.pi*sigma_reward**2)**0.1
    return prob_per_sample

In [ ]:
plt.hist(np.array(llh_under(data['z'], data['r'], alpha=90, sigma_reward=.5, method='np')))

In [ ]:
np.prod(llh_under(data['z'], data['r'], alpha=90, sigma_reward=.5))

In [ ]:
N=1000

def int_over_prior(alpha_samples):
    return np.mean(np.array([helper.model_llh(data['z'], data['r'], alpha=alpha_sample, sigma_reward=.5, method='np') for alpha_sample in alpha_samples]))

# 2D model
a = [tfd.Uniform(0,180).sample(N),np.repeat(90,N),np.repeat(0,N),np.repeat(45,N)]
labels = ('2D','90','0','45')
plt.bar(labels,[np.log(int_over_prior(alpha_samples)) for alpha_samples in a])

In [ ]:
alpha_gt = 45
data = helper.generate_data(2,alpha=alpha_gt,sigma_reward=0.001)
alphas = np.arange(0, 360, 5)
p_gamma = np.array([helper.model_llh(data['z'], data['r'], alpha=alpha, sigma_reward=1) for alpha in alphas])
plt.plot(alphas,p_gamma)
plt.axvline(x=alpha_gt)


In [ ]:
alpha_gt = 90
data = helper.generate_data(2,alpha=alpha_gt,sigma_reward=0.001)
alphas = np.arange(0, 360, 5)
p_gamma = np.array([helper.model_llh(data['z'], data['r'], alpha=alpha, sigma_reward=1) for alpha in alphas])
plt.plot(alphas,p_gamma)
#plt.gca().set_ylim([-100,1000])
plt.axvline(x=alpha_gt)

In [ ]:
def model_llh_gamma(z, r, gamma, sigma_reward):
    prob_per_sample = tfp.distributions.Normal(loc=tf.reduce_sum(tf.multiply(gamma,z),1), scale=sigma_reward).prob(r)
    return tf.reduce_prod(prob_per_sample)

def model_llh2_gamma(z, r, gamma, sigma_reward):
    prob_per_sample = tf.exp(-0.5 * (tf.reduce_sum(tf.multiply(gamma,z),1) - r)**2 / sigma_reward**2) / (2*np.pi*sigma_reward**2)**0.5
    return tf.reduce_prod(prob_per_sample)

In [ ]:
alpha_gt = 90
data = helper.generate_data(2,alpha=alpha_gt,sigma_reward=0.001)

#x = np.arange(0, 1, 0.1)
#y = np.arange(0, 1, 0.1)
p_gamma = np.array([model_llh_gamma(data['z'], data['r'], gamma=np.array([0.,1.]).astype(np.float32), sigma_reward=0.1) for alpha in alphas])

plt.plot(alphas,p_gamma)


In [ ]:
p_gamma

In [ ]:
xx,yy

In [ ]:
yy

In [ ]:
x = np.arange(0, 1, 0.1)
y = np.arange(0, 1, 0.1)
xx, yy = np.meshgrid(x, y, sparse=True)

In [ ]:
z = np.sin(xx**2 + yy**2) / (xx**2 + yy**2)

In [ ]:
x = np.arange(-5, 5, 0.1)
y = np.arange(-5, 5, 0.1)
xx, yy = np.meshgrid(x, y, sparse=True)
zz = model_llh_gamma(data['z'], data['r'], gamma=np.array([xx,yy]).astype(np.float32), sigma_reward=0.1)
h = plt.contourf(x,y,z)
plt.show()

In [ ]:
np.array([xx,yy])